# LLM(Large Language Model)의 기초 이해

이 노트북은 LLM이 **문자열을 어떻게 인식하고**, "작업"이라는 것을 **어떻게 수행하는지**를 단계별로 시각화합니다.

## 전체 흐름
```
텍스트 입력 → ① 토큰화 → ② 임베딩 → ③ Attention → ④ 다음 토큰 예측 → 텍스트 출력
```

**환경**: `conda activate agent`

In [ ]:
# 필수 라이브러리 임포트
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import font_manager
from transformers import AutoTokenizer, GPT2Model, GPT2LMHeadModel
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
def setup_korean_font():
    import subprocess
    try:
        result = subprocess.run(
            ['fc-match', '-f', '%{file}', 'Noto Sans CJK KR'],
            capture_output=True, text=True, timeout=5)
        font_path = result.stdout.strip()
        if font_path and os.path.exists(font_path):
            font_manager.fontManager.addfont(font_path)
            prop = font_manager.FontProperties(fname=font_path)
            plt.rcParams['font.family'] = prop.get_name()
            print(f"[INFO] 한글 폰트: {prop.get_name()} ({font_path})")
            plt.rcParams['axes.unicode_minus'] = False
            return
    except Exception:
        pass
    for name in ['NanumGothic', 'Noto Sans CJK KR', 'Noto Sans CJK JP']:
        if name in [f.name for f in font_manager.fontManager.ttflist]:
            plt.rcParams['font.family'] = name
            print(f"[INFO] 한글 폰트 (fallback): {name}")
            break
    plt.rcParams['axes.unicode_minus'] = False

setup_korean_font()
plt.rcParams['figure.dpi'] = 120

print(f'PyTorch: {torch.__version__}')
print(f'CUDA: {torch.cuda.is_available()}')
print('Ready!')

---
## 1단계: 토큰화 (Tokenization)

LLM은 문자열을 직접 이해할 수 없습니다. 먼저 텍스트를 **토큰(token)**이라는 작은 단위로 쪼갭니다.

GPT-2는 **BPE(Byte Pair Encoding)** 방식을 사용합니다:
1. 모든 문자를 개별 토큰으로 시작
2. 가장 빈번한 인접 쌍을 반복적으로 병합
3. 최종적으로 약 50,257개의 어휘(vocabulary) 구축

In [ ]:
# GPT-2 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# 다양한 문장 토큰화 해보기
sentences = [
    "Hello, how are you?",
    "The transformer model is powerful.",
    "인공지능은 미래를 바꿀 것입니다.",
    "unhappiness",  # 형태소가 어떻게 분리되는지 관찰
]

for sent in sentences:
    ids = tokenizer.encode(sent)
    tokens = tokenizer.convert_ids_to_tokens(ids)
    print(f'\n원본: "{sent}"')
    print(f'토큰: {tokens}')
    print(f'토큰 ID: {ids}')
    print(f'토큰 수: {len(ids)}')

In [ ]:
# 토큰화 시각화
text = "AI changes the world"
ids = tokenizer.encode(text)
tokens = tokenizer.convert_ids_to_tokens(ids)

fig, ax = plt.subplots(figsize=(14, 3))
ax.axis('off')
ax.set_title(f'토큰화: "{text}"', fontsize=14, fontweight='bold')

cmap = plt.cm.get_cmap('Pastel1')
n = len(tokens)
for i, (tok, tid) in enumerate(zip(tokens, ids)):
    x = 0.05 + i * 0.18
    rect = mpatches.FancyBboxPatch(
        (x, 0.3), 0.15, 0.4,
        boxstyle='round,pad=0.02', facecolor=cmap(i/max(n-1,1)),
        edgecolor='black', linewidth=2, transform=ax.transAxes)
    ax.add_patch(rect)
    ax.text(x+0.075, 0.55, tok.replace('Ġ', '_ '),
            fontsize=12, ha='center', va='center', fontweight='bold',
            transform=ax.transAxes)
    ax.text(x+0.075, 0.38, f'ID: {tid}',
            fontsize=9, ha='center', va='center', color='gray',
            transform=ax.transAxes)

plt.tight_layout()
plt.show()

---
## 2단계: 임베딩 (Embedding)

토큰 ID는 단순한 정수입니다. LLM은 이를 **고차원 벡터(임베딩)**로 변환합니다.

- GPT-2: 각 토큰 → **768차원** 벡터
- 의미적으로 유사한 단어는 벡터 공간에서 **가까이** 위치
- 이 벡터가 Transformer 모델의 실제 입력이 됩니다

In [ ]:
# GPT-2 모델 로드
model = GPT2Model.from_pretrained("gpt2")
model.eval()

# 단어들의 임베딩 추출
words = ["king", "queen", "man", "woman", "prince", "princess",
         "dog", "cat", "car", "bicycle", "happy", "sad"]

embeddings = []
with torch.no_grad():
    for w in words:
        ids = tokenizer.encode(w)
        emb = model.wte(torch.tensor([ids[0]])).squeeze().numpy()
        embeddings.append(emb)

embeddings = np.array(embeddings)
print(f'임베딩 크기: {embeddings.shape}  (단어 수 × 차원)')

In [ ]:
# 코사인 유사도 히트맵 + PCA 2D 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# 히트맵
sim = cosine_similarity(embeddings)
ax = axes[0]
im = ax.imshow(sim, cmap='RdYlBu_r', vmin=-0.3, vmax=0.5)
ax.set_xticks(range(len(words)))
ax.set_yticks(range(len(words)))
ax.set_xticklabels(words, rotation=45, ha='right', fontsize=9)
ax.set_yticklabels(words, fontsize=9)
ax.set_title('코사인 유사도 히트맵', fontsize=13, fontweight='bold')
for i in range(len(words)):
    for j in range(len(words)):
        ax.text(j, i, f'{sim[i,j]:.2f}', ha='center', va='center', fontsize=6)
fig.colorbar(im, ax=ax, shrink=0.8)

# PCA 2D
ax2 = axes[1]
pca = PCA(n_components=2)
coords = pca.fit_transform(embeddings)
categories = {'royalty': [0,1,4,5], 'gender': [2,3], 'animal': [6,7],
              'vehicle': [8,9], 'emotion': [10,11]}
cat_colors = {'royalty': 'red', 'gender': 'blue', 'animal': 'green',
              'vehicle': 'orange', 'emotion': 'purple'}
for cat, idxs in categories.items():
    ax2.scatter(coords[idxs, 0], coords[idxs, 1],
               c=cat_colors[cat], s=120, label=cat, zorder=5)
    for i in idxs:
        ax2.annotate(words[i], coords[i], textcoords='offset points',
                     xytext=(6, 6), fontsize=10, fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)
ax2.set_title('PCA 2D - 의미 공간에서의 단어 위치', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

---
## 3단계: Self-Attention

Transformer의 핵심! 각 토큰이 **다른 모든 토큰과의 관계**를 계산합니다.

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) \cdot V$$

- **Query(Q)**: "나는 무엇을 찾고 있는가?" — 질의
- **Key(K)**: "나는 어떤 정보를 가지고 있는가?" — 라벨
- **Value(V)**: "내가 제공할 실제 정보" — 내용

In [ ]:
# 실제 GPT-2 Attention 가중치 추출
model_attn = GPT2Model.from_pretrained("gpt2", output_attentions=True)
model_attn.eval()

text = "The cat sat on the mat"
inputs = tokenizer(text, return_tensors='pt')
tokens_list = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

with torch.no_grad():
    out = model_attn(**inputs)

# Layer 0, 4개 Head의 Attention 가중치
attn_layer0 = out.attentions[0][0].numpy()

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
fig.suptitle(f'GPT-2 Attention 가중치 (Layer 0)\n"{text}"',
             fontsize=14, fontweight='bold')

for h, ax in enumerate(axes.flat):
    im = ax.imshow(attn_layer0[h], cmap='Blues', vmin=0)
    ax.set_xticks(range(len(tokens_list)))
    ax.set_yticks(range(len(tokens_list)))
    ax.set_xticklabels([t.replace('Ġ', '_ ') for t in tokens_list],
                       rotation=45, ha='right', fontsize=9)
    ax.set_yticklabels([t.replace('Ġ', '_ ') for t in tokens_list], fontsize=9)
    ax.set_title(f'Head {h}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Key')
    ax.set_ylabel('Query')
    fig.colorbar(im, ax=ax, shrink=0.6)

plt.tight_layout()
plt.show()

---
## 4단계: 텍스트 생성 (다음 토큰 예측)

LLM의 "작업 수행" = **다음 토큰 예측의 반복**

1. 입력 텍스트를 토큰화
2. 모델이 다음 토큰의 **확률 분포** 출력 (50,257개 어휘 전체)
3. 샘플링 전략으로 하나의 토큰 선택
4. 선택된 토큰을 입력에 추가 → 2번으로 돌아감 (자기회귀)

In [ ]:
# 다음 토큰 예측 확률 분포
lm_model = GPT2LMHeadModel.from_pretrained("gpt2")
lm_model.eval()

prompt = "The capital of France is"
inputs = tokenizer(prompt, return_tensors='pt')

with torch.no_grad():
    logits = lm_model(**inputs).logits[0, -1, :]
    probs = torch.softmax(logits, dim=-1)

top_k = 15
top_probs, top_idx = torch.topk(probs, top_k)
top_tokens = [tokenizer.decode([i]).strip() for i in top_idx]

fig, ax = plt.subplots(figsize=(12, 6))
colors = plt.cm.RdYlGn(np.linspace(0.8, 0.2, top_k))
bars = ax.barh(range(top_k-1, -1, -1), top_probs.numpy(), color=colors)
ax.set_yticks(range(top_k-1, -1, -1))
ax.set_yticklabels([f'"{t}"' for t in top_tokens], fontsize=11)
ax.set_xlabel('확률', fontsize=12)
ax.set_title(f'다음 토큰 예측: "{prompt} ___"', fontsize=14, fontweight='bold')
for i, (bar, p) in enumerate(zip(bars, top_probs)):
    ax.text(bar.get_width() + 0.005, top_k-1-i, f'{p:.3f} ({p*100:.1f}%)',
            va='center', fontsize=9)
plt.tight_layout()
plt.show()

In [ ]:
# Temperature 비교
raw_logits = logits.numpy()

def softmax_temp(logits, T):
    scaled = logits / T
    e = np.exp(scaled - np.max(scaled))
    return e / e.sum()

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Temperature가 확률 분포에 미치는 영향', fontsize=14, fontweight='bold')

for ax, T in zip(axes, [0.3, 1.0, 2.0]):
    p = softmax_temp(raw_logits, T)
    top_p = np.sort(p)[-15:][::-1]
    top_i = np.argsort(p)[-15:][::-1]
    top_t = [tokenizer.decode([i]).strip() for i in top_i]
    
    ax.barh(range(14, -1, -1), top_p, color=plt.cm.RdYlGn(np.linspace(0.8, 0.2, 15)))
    ax.set_yticks(range(14, -1, -1))
    ax.set_yticklabels([f'"{t}"' for t in top_t], fontsize=8)
    ax.set_title(f'T={T}' + (' (확정적)' if T < 1 else ' (기본)' if T == 1 else ' (창의적)'),
                fontsize=12, fontweight='bold')
    ax.set_xlim(0, max(top_p) * 1.2)

plt.tight_layout()
plt.show()

In [ ]:
# 자기회귀 생성 과정 시각화
prompt = "Once upon a time"
input_ids = tokenizer.encode(prompt, return_tensors='pt')
current_ids = input_ids.clone()

print(f'초기 입력: "{prompt}"\n')
print('=' * 60)

for step in range(8):
    with torch.no_grad():
        logits = lm_model(current_ids).logits[0, -1, :]
    probs = torch.softmax(logits, dim=-1)
    top5_p, top5_i = torch.topk(probs, 5)
    top5_t = [tokenizer.decode([i]).strip() for i in top5_i]
    
    # Greedy: 최고 확률 토큰 선택
    next_id = top5_i[0].unsqueeze(0).unsqueeze(0)
    current_ids = torch.cat([current_ids, next_id], dim=-1)
    
    current_text = tokenizer.decode(current_ids[0])
    candidates = ' | '.join([f'"{t}"({p:.1%})' for t, p in zip(top5_t, top5_p)])
    print(f'Step {step+1}: 후보 → {candidates}')
    print(f'         선택: "{top5_t[0]}"  →  현재: "{current_text}"')
    print()

print('=' * 60)
print(f'최종 결과: "{tokenizer.decode(current_ids[0])}"')

---
## 핵심 정리

| 단계 | 설명 | 수학적 표현 |
|------|------|------------|
| **토큰화** | 텍스트 → 정수 ID 시퀀스 | `"Hello" → [15496]` |
| **임베딩** | 정수 ID → 고차원 벡터 | `15496 → [0.12, -0.34, ...]` (768차원) |
| **Attention** | 토큰 간 관계 계산 | `softmax(QK^T/√d) · V` |
| **출력 생성** | 다음 토큰 확률 예측 | `P(next_token \| context)` |

**LLM이 "작업을 수행한다"는 것 = 적절한 다음 토큰을 반복적으로 예측하는 것**

학습 과정에서 방대한 텍스트를 통해 언어의 패턴, 사실 관계, 추론 능력 등을 임베딩과 Attention 가중치에 압축 저장합니다.